# Spark

25.08.01(금) 12주57일차

In [14]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TestLocal") \
    .master("local[*]") \
    .getOrCreate()

spark.range(10).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



25/08/01 12:29:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [15]:
spark.stop()

# 도커, 파이스파크 실습 환경 구성

# **1. 기본 Dockerfile 구성**

작업 디렉토리 생성

PySpark와 Jupyter Notebook을 포함하는 Dockerfile을 만든다.

기존 도커 이미지로 설치하기.

In [16]:
# pyspark-notebook 폴더가 없으면 생성합니다.
!mkdir -p pyspark-notebook


In [17]:
%%writefile ./pyspark-notebook/Dockerfile

FROM jupyter/pyspark-notebook

USER root

RUN apt-get update && apt-get install -y vim tree net-tools

USER jovyan

Overwriting ./pyspark-notebook/Dockerfile


In [18]:
!ls -l pyspark-notebook/

total 4
-rw-rw-r-- 1 devuser devuser 116 Aug  1 12:29 Dockerfile


In [19]:
!docker build -t pyspark-custom-notebook ./pyspark-notebook/

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 153B                                       0.0s
 => [internal] load metadata for docker.io/jupyter/pyspark-notebook:lates  0.1s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 153B                                       0.0s
 => [internal] load metadata for docker.io/jupyter/pyspark-notebook:lates  0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 153B                                       0.0s
 => [internal] load metadata for docker.

- jupyter/pyspark-notebook 이미지
    - `jupyter/pyspark-notebook`: 공식적으로 제공되는 PySpark + Jupyter 통합 이미지
    - **기반(Base image)** 으로 삼아 Dockerfile에서  설정(예: `vim`, `tree` 설치)을 추가한다.

참고 : jdk 이미지 위에 스파크 설치 하는 이미지 만들기

In [20]:
%%writefile Dockerfile

FROM openjdk:11

ENV SPARK_VERSION=3.5.0 \
    HADOOP_VERSION=3 \
    PYTHON_VERSION=3.10

RUN apt update && apt install -y \
    curl python${PYTHON_VERSION} python3-pip && \
    pip3 install jupyter findspark pandas matplotlib

# Spark 설치
RUN curl -L https://downloads.apache.org/spark/spark-${SPARK_VERSION}/spark-${SPARK_VERSION}-bin-hadoop${HADOOP_VERSION}.tgz \
    | tar -xz -C /opt && \
    ln -s /opt/spark-${SPARK_VERSION}-bin-hadoop${HADOOP_VERSION} /opt/spark

ENV SPARK_HOME=/opt/spark \
    PATH=$PATH:/opt/spark/bin \
    PYSPARK_PYTHON=python3 \
    PYSPARK_DRIVER_PYTHON=jupyter \
    PYSPARK_DRIVER_PYTHON_OPTS="notebook --ip='0.0.0.0' --port=8888 --no-browser --allow-root"

WORKDIR /workspace
EXPOSE 8888

CMD ["pyspark"]

Overwriting Dockerfile


In [21]:
!ls -l

total 624
-rw-rw-r-- 1 devuser devuser 621297 Aug  1 12:29 01_start_spark_RDD.ipynb
-rw-rw-r-- 1 devuser devuser   2528 Jul 31 17:33 25.07.31_도커_mySQL.ipynb
drwxrwxr-x 4 devuser devuser   4096 Aug  1 12:29 artifacts
-rw-rw-r-- 1 devuser devuser    747 Aug  1 12:30 Dockerfile
drwxrwxr-x 2 devuser devuser   4096 Aug  1 12:16 pyspark-notebook


# 25.08.01 오전수업

In [22]:
print('hello')

hello


In [23]:
!which python

In [24]:
!python3 --version

Python 3.10.12


In [25]:
!which java

/bin/java


In [26]:
!python --version

/bin/bash: line 1: python: command not found


In [27]:
!java -version

openjdk version "17.0.15" 2025-04-15
OpenJDK Runtime Environment (build 17.0.15+6-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 17.0.15+6-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [28]:
!pip show pyspark

Name: pyspark
Version: 4.0.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /home/devuser/.local/lib/python3.10/site-packages
Requires: py4j
Required-by: 


In [29]:
#SPARK_HOME
import os
os.environ.get('SPARK_HOME')

In [30]:
#JAVA_HOME
os.environ.get('JAVA_HOME')


In [31]:
#PYTHONPATH
os.environ.get('PYTHONPATH')

In [32]:
import pyspark

In [33]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark example1').getOrCreate() #chaining
#SparkContext.SparkSession

25/08/01 12:30:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [34]:
spark

In [35]:
spark.stop()

In [36]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate() #chaining

25/08/01 12:30:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [37]:
spark

In [38]:
data = [('Alice',1), ('Bob', 2), ('Charlie',3) ]
type(data)

list

In [39]:
#DataFrame 객체(분산객체)를 생성 <> 판다스의 데이터프레임이 아님.
data1 = spark.createDataFrame( data, ['Name','Value'])
data1[1][1]

Column<'Value[1]'>

In [40]:
data1.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



# RDD 객체 생성

In [41]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('pyspark example1').getOrCreate()

In [42]:
from pyspark.sql import SparkSession

# 1. spark 변수(SparkSession)를 먼저 생성합니다.
spark = SparkSession.builder.appName("rdd_example").getOrCreate()

# 2. 생성된 spark 변수를 사용해서 RDD를 만듭니다.
rdd = spark.sparkContext.parallelize([1,2,3,4,5])
rdd

25/08/01 12:30:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


ParallelCollectionRDD[7] at readRDDFromFile at PythonRDD.scala:297

In [43]:
from pyspark.sql import SparkSession

# 1. SparkSession을 생성합니다.
spark = SparkSession.builder.appName("data1_example").getOrCreate()

# 2. 파이썬 리스트 데이터를 준비합니다.
data = [('Alice', 1), ('Bob', 2), ('Charlie', 3)]

# 3. spark.createDataFrame으로 data1 변수를 생성합니다.
data1 = spark.createDataFrame(data, ['Name', 'Value'])

# 4. 이제 data1을 호출할 수 있습니다.
data1.show()

25/08/01 12:30:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [44]:
rdd.take(5) #rdd 객체를 출력하는 함수 - n개를 지정

[1, 2, 3, 4, 5]

In [45]:
#map 연산 : rdd 값으로 연산
squared_rdd = rdd.map(lambda x:x*x)
squared_rdd

PythonRDD[17] at RDD at PythonRDD.scala:56

In [46]:
rdd.take(3)

[1, 2, 3]

In [47]:
squared_rdd.collect()

[1, 4, 9, 16, 25]